# Preparing Insight data for PIConGPU

## Intro
This notebook allows you to manipulate Insight data and prepare it to be read via the InsightPulse profile into PIconGPU.
The raw Insight data cannot be used, since it (at least) has to be phase corrected and transformed into the time domain. 
Furthermore, the field can be propagated.

## Load modules

In [ ]:
# standard modules
import numpy as np
import matplotlib.pyplot as plt

# modules from preparingInsightData.py
from preparingInsightData import preproutines

## Get the data
The far field data from an Insight measurement is stored in a h5 file and typically measured in dependence of two transversal coordinates ("x" and "y" in mm) and the frequency ("w" in rad/fs). If there are any deviations from this scheme, you will have to adjust the _preproutines_ source code. 

Then, the first steps are:
1. read the far field data and store it in numpy arrays
2. fit the far field data intensity with a 2D gaussian to extract the beam center and waist size
3. propagate to the near field
4. fit the near field data with a 2D supergaussian to extract the beam center and waist size

For that, we provide path, filename and focal distance (same unit as the transversal scales) to the init funcion.

In [ ]:
insight = preproutines("/put/your/path/here/", "filename.h5", 2000)

In [ ]:
# some nice colorful pictures of our data
fig = plt.figure(figsize=(14, 8))

ax1 = fig.add_subplot(231)
ax1.imshow(
    np.sum(np.abs(insight.Ew), axis=-1),
    cmap="cubehelix",
    origin="lower",
    extent=(insight.x[0], insight.x[-1], insight.y[0], insight.y[-1]),
)
ax1.set_title("spectral integrated amplitude, far field")
ax1.set_xlabel("x [mm]")
ax1.set_ylabel("y [mm]")
ax1.set_xlim(-0.2, 0.2)
ax1.set_ylim(-0.2, 0.2)

ax2 = fig.add_subplot(232)
ax2.imshow(
    np.sum(np.abs(insight.Ew_NF), axis=-1),
    cmap="cubehelix",
    origin="lower",
    extent=(insight.x_NF[0], insight.x_NF[-1], insight.y_NF[0], insight.y_NF[-1]),
)
ax2.set_title("spectral integrated amplitude, near field")
ax2.set_xlabel("x [mm]")
ax2.set_ylabel("y [mm]")
ax2.set_xlim(-100, 100)
ax2.set_ylim(-100, 100)

ax3 = fig.add_subplot(233)
ax3.plot(
    insight.w,
    np.angle(
        insight.Ew_NF[np.abs(insight.y_NF - insight.yc_NF).argmin(), np.abs(insight.x_NF - insight.xc_NF).argmin(), :]
    ),
)
ax3.set_title("phase in near field beam center")
ax3.set_xlabel(r"$\omega$ [rad/fs]")
ax3.set_ylabel("rad/fs")

ax4 = fig.add_subplot(234)
ax4.imshow(
    np.sum(np.abs(insight.Ew), axis=0),
    cmap="cubehelix",
    origin="lower",
    aspect="auto",
    extent=(insight.w[0], insight.w[-1], insight.x[0], insight.x[-1]),
)
ax4.set_title(r"SD$_x$ in focus")
ax4.set_ylabel("x [mm]")
ax4.set_xlabel(r"$\omega$ [rad/fs]")
ax4.set_ylim(-0.2, 0.2)

ax5 = fig.add_subplot(235)
ax5.imshow(
    np.sum(np.abs(insight.Ew), axis=1),
    cmap="cubehelix",
    origin="lower",
    aspect="auto",
    extent=(insight.w[0], insight.w[-1], insight.x[0], insight.x[-1]),
)
ax5.set_title(r"SD$_y$ in focus")
ax5.set_xlabel(r"$\omega$ [rad/fs]")
ax5.set_ylabel("y [mm]")
ax5.set_ylim(-0.2, 0.2)

ax6 = fig.add_subplot(236)
# sum just over the main beam spot to extract the spectrum
ax6.plot(
    insight.w,
    np.sum(
        np.sum(
            np.abs(
                insight.Ew[
                    np.abs(insight.y - insight.yc + 2 * insight.waist).argmin() : np.abs(
                        insight.y - insight.yc - 2 * insight.waist
                    ).argmin(),
                    np.abs(insight.x - insight.xc + 2 * insight.waist).argmin() : np.abs(
                        insight.x - insight.xc - 2 * insight.waist
                    ).argmin(),
                    :,
                ]
            )
            ** 2,
            axis=0,
        ),
        axis=0,
    ),
)
ax6.set_title("spectral intensity")
ax6.set_xlabel(r"$\omega$ [rad/fs]")

plt.subplots_adjust(hspace=0.3, wspace=0.3)
plt.show()

## Correct the data
### Adjust beam compression and add dispersion parameters
Before we go on with any calculations, we have to correct the phase. Insight reconstructs the amplitude of the far field beam aswell as the phase, up to a unknown global phase for every frequency.
For an estimation of this global phase, we assume perfect compression in the (near field) beam center. Thus, we extract the phase in the beam center in dependence of the frequency and substract this function globally (i.e. from the measured phase in dependence of the frequency at every space point).
Here, we also have the possibility to add dispersion parameters such as group velocity dispersion (GVD) and third order dispersion (TOD) (both are set to 0 by default).

In [ ]:
insight.correctPhase()

In [ ]:
plt.plot(
    insight.w,
    np.angle(
        insight.Ew_NF[np.abs(insight.y_NF - insight.yc_NF).argmin(), np.abs(insight.x_NF - insight.xc_NF).argmin(), :]
    ),
)
plt.title("corrected phase in near field beam center")
plt.xlabel(r"$\omega$ [rad/fs]")
plt.ylabel("rad/fs")
plt.show()

### Correct ugly spots in the near field (optional)
Sometimes, the amplitude in the near field looks weird, showing some (unphysical) peaks or holes. These can cause artefacts in the far field and will thus be smoothened out.

In [ ]:
insight.correctUglySpotInNF(0.0, 0.0)  # x-coordinate, y-coordinate of ugly spot

In [ ]:
fig = plt.figure(figsize=(10, 4))

ax1 = fig.add_subplot(121)
ax1.imshow(
    np.sum(np.abs(insight.Ew), axis=-1),
    cmap="cubehelix",
    origin="lower",
    extent=(insight.x[0], insight.x[-1], insight.y[0], insight.y[-1]),
)
ax1.set_title("spectral integrated amplitude, far field")
ax1.set_xlabel("x [mm]")
ax1.set_ylabel("y [mm]")
ax1.set_xlim(-0.2, 0.2)
ax1.set_ylim(-0.2, 0.2)

ax2 = fig.add_subplot(122)
ax2.imshow(
    np.sum(np.abs(insight.Ew_NF), axis=-1),
    cmap="cubehelix",
    origin="lower",
    extent=(insight.x_NF[0], insight.x_NF[-1], insight.y_NF[0], insight.y_NF[-1]),
)
ax2.set_title("spectral integrated amplitude, near field")
ax2.set_xlabel("x [mm]")
ax2.set_ylabel("y [mm]")
ax2.set_xlim(-100, 100)
ax2.set_ylim(-100, 100)

plt.show()

### Center the near field beam spot (optional)
When the near field beam spot is not centered (please check the center coordinates above), the far field will propagate obliquely instead of straight ahead. Centering the near field prevents this.

In [ ]:
insight.shiftNFtoCenter()

In [ ]:
fig = plt.figure(figsize=(14, 4))

ax1 = fig.add_subplot(131)
ax1.imshow(
    np.sum(np.abs(insight.Ew), axis=-1),
    cmap="cubehelix",
    origin="lower",
    extent=(insight.x[0], insight.x[-1], insight.y[0], insight.y[-1]),
)
ax1.set_title("spectral integrated amplitude, far field")
ax1.set_xlabel("x [mm]")
ax1.set_ylabel("y [mm]")
ax1.set_xlim(-0.2, 0.2)
ax1.set_ylim(-0.2, 0.2)

ax2 = fig.add_subplot(132)
ax2.imshow(
    np.sum(np.abs(insight.Ew_NF), axis=-1),
    cmap="cubehelix",
    origin="lower",
    extent=(insight.x_NF[0], insight.x_NF[-1], insight.y_NF[0], insight.y_NF[-1]),
)
ax2.set_title("spectral integrated amplitude, near field")
ax2.set_xlabel("x [mm]")
ax2.set_ylabel("y [mm]")
ax2.set_xlim(-100, 100)
ax2.set_ylim(-100, 100)
plt.show()

## Propagate
Now the far field data is ready to be propagated. For that, the angular spectrum method is used. 
Watch out not to propagate too far, since then the growing beam diameter could reach the transversal window borders and thus cause fourier transform artefacts.

In [ ]:
z = -2  # mm
Ew_prop = insight.propagate(z)

In [ ]:
plt.imshow(
    np.sum(np.abs(Ew_prop), axis=-1),
    cmap="cubehelix",
    origin="lower",
    extent=(insight.x[0], insight.x[-1], insight.y[0], insight.y[-1]),
)
plt.title("spectral integrated amplitude, propagated to %.2f mm" % (z))
plt.xlabel("x [mm]")
plt.ylabel("y [mm]")
# plt.xlim(-.2, .2)
# plt.ylim(-.2, .2)
plt.show()

## Transform to the time domain
The far field data will be transformed to the time domain via a 1D fourier transformation. This takes a while, since the spectrum has to be extended and the field data extrapolated. One can adjust the number of samples per wavelength, which is set to 10 by default.

In [ ]:
insight.toTimeDomain(Ew_prop)

In [ ]:
fig = plt.figure(figsize=(7, 7))

ax1 = fig.add_subplot(211)
ax1.imshow(
    np.sum(np.abs(insight.Et), axis=0),
    cmap="cubehelix",
    origin="lower",
    aspect="auto",
    extent=(insight.t[0], insight.t[-1], insight.x[0], insight.x[-1]),
)
ax1.set_ylabel("x [mm]")
ax1.set_xlim(-100, 100)
ax1.set_ylim(-0.2, 0.2)

ax2 = fig.add_subplot(212)
ax2.imshow(
    np.sum(np.abs(insight.Et), axis=1),
    cmap="cubehelix",
    origin="lower",
    aspect="auto",
    extent=(insight.t[0], insight.t[-1], insight.x[0], insight.x[-1]),
)
ax2.set_xlabel("t [fs]")
ax2.set_ylabel("y [mm]")
ax2.set_xlim(-100, 100)
ax2.set_ylim(-0.2, 0.2)

plt.show()

## Save data to OpenPMD
The data is now nearly ready te be used as InsightPulse input. We still have to correct the amplitude of the pulse in the time domain (= scaling it to the actual beam energy, given in Joule) and save it to an OpenPMD checkpoint. Furthermore, the field data chunk may still be too big: the whole chunk will be stored on the GPUs afterwards, but their memory is limited.
The following function corrects the amplitude by scaling to the actual beam enregy and allows cropping the borders of the field data chunk before saving it at the provided destination path.

In [ ]:
insight.saveToOpenPMD("", "insightData_300um%T.h5", energy=4.5, crop_x=0.3, crop_y=0.3, crop_t=100)